# Pre-Trained Models

In this notebook, pretrained model (EfficientNetV2B0) will be implemented on the train dataset.

In [1]:
from keras.models import load_model
# imports
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import tensorflow as tf
import seaborn as sns
import sys
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import (Dense, Dropout, Flatten, 
                                     Conv2D, MaxPooling2D, BatchNormalization, 
                                     GlobalAveragePooling2D)
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras import metrics
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.layers.experimental.preprocessing import (RandomFlip, RandomRotation, 
                                                                RandomZoom, RandomContrast)
# For reproducibility
np.random.seed(42)
# Source: https://www.tensorflow.org/api_docs/python/tf/keras/utils/set_random_seed
tf.keras.utils.set_random_seed(42)

In [13]:
# Importing global variables
sys.path.append('../../Code/Helper')
import helper as hp

In [3]:
base_path = '../../Data/Final'
w = 48
h = 48
# Flow from directory
# Code modified from: Lesson 8.06-CNN
train = image_dataset_from_directory(
    base_path + '/train',
    image_size=(w,h),
    batch_size=32,
    seed=42,
    validation_split = 0.2,
    subset= 'training',
    label_mode='categorical'
)

val = image_dataset_from_directory(
    base_path + '/train',
    image_size=(w,h),
    batch_size=32,
    seed=42,
    validation_split = 0.2,
    subset= 'validation',
    label_mode='categorical'
)
    
test = image_dataset_from_directory(
    base_path + '/test',
    image_size=(w,h),
    batch_size=32,
    label_mode='categorical',
    shuffle= False
)

Found 50499 files belonging to 7 classes.
Using 40400 files for training.
Found 50499 files belonging to 7 classes.
Using 10099 files for validation.
Found 7178 files belonging to 7 classes.


## EfficientNetV2B0

In [4]:
effnet = Sequential()

effnet_pretrained = EfficientNetV2B0(
    input_shape= (h,w,3),
    weights='imagenet',    
    include_top=False,
    include_preprocessing = True,
    classifier_activation='softmax'
    )


effnet_pretrained.trainable = True

effnet.add(effnet_pretrained)

effnet.add(GlobalAveragePooling2D())
effnet.add(Dense(128, activation='relu'))
effnet.add(BatchNormalization())
effnet.add(Dropout(0.5))

effnet.add(Dense(7, activation='softmax'))
effnet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-b0 (Function  (None, 2, 2, 1280)       5919312   
 al)                                                             
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                        

In [5]:
# Compile
effnet.compile(
    optimizer= 'adam', 
    loss = 'categorical_crossentropy',
    metrics = ['acc',
               metrics.Precision(),
               metrics.Recall(),
               metrics.AUC()]
)

In [6]:
es = EarlyStopping(patience = 10,
                   monitor = 'val_loss',
                   mode = 'min')

lr = ReduceLROnPlateau(monitor = 'val_loss',
                       factor = 0.3,
                       patience = 2,
                       min_delta = 0.0001,
                       verbose = 1)

ck = ModelCheckpoint('../../Models/effnet.h5',
                    save_best_only = True,
                    monitor = 'val_acc',
                    mode = 'max')

In [9]:
effnet_hist = effnet.fit(train, 
                validation_data = val,
                epochs = 50,
                callbacks = [es, lr, ck],
                verbose = 1)

Epoch 1/50
1263/1263 [==============================] - 138s 109ms/step - loss: 1.3322 - acc: 0.4896 - precision: 0.7264 - recall: 0.2902 - auc: 0.8447 - val_loss: 1.1139 - val_acc: 0.5697 - val_precision: 0.8081 - val_recall: 0.3549 - val_auc: 0.8959 - lr: 0.0010
Epoch 2/50
1263/1263 [==============================] - 132s 105ms/step - loss: 1.1309 - acc: 0.5700 - precision: 0.7428 - recall: 0.4007 - auc: 0.8911 - val_loss: 1.0569 - val_acc: 0.5908 - val_precision: 0.7854 - val_recall: 0.4049 - val_auc: 0.9056 - lr: 0.0010
Epoch 3/50
1263/1263 [==============================] - 132s 105ms/step - loss: 1.0270 - acc: 0.6137 - precision: 0.7555 - recall: 0.4704 - auc: 0.9111 - val_loss: 1.0505 - val_acc: 0.6011 - val_precision: 0.7410 - val_recall: 0.4647 - val_auc: 0.9068 - lr: 0.0010
Epoch 4/50
1263/1263 [==============================] - 132s 104ms/step - loss: 0.9373 - acc: 0.6519 - precision: 0.7689 - recall: 0.5279 - auc: 0.9262 - val_loss: 1.0720 - val_acc: 0.6006 - val_precision:

The EfficientnetV2B0 model is the smallest Efficientnet Version 2 model. The model weights were initialized with `imagenet` weights that are given by the keras API. While the training loss and training accuracy were promising (low train loss of 0.1034 and high train accuracy of 96.60%) the validation scores depict the model is not performing so well on the validation set. The loss was much higher with 1.6830 and the validation accuracy was 62.74%. This is lower than our best performing custom CNN model, which has a Validation Accuracy score of 64.46%. 

# EfficientNet model 2

In [ ]:
effnet_2 = Sequential()

effnet_pretrained = EfficientNetV2B0(
    input_shape= (h,w,3),
    weights='imagenet',    
    include_top=False,
    include_preprocessing = True,
    classifier_activation='softmax'
    )


effnet_pretrained.trainable = True

effnet_2.add(data_augmentation)
effnet_2.add(effnet_pretrained)

effnet_2.add(GlobalAveragePooling2D())

effnet_2.add(Dense(256, activation='relu'))
effnet_2.add(BatchNormalization())
effnet_2.add(Dropout(0.5))

effnet_2.add(Dense(7, activation='softmax'))
effnet_2.summary()

In [290]:
# Compile
effnet_2.compile(
    optimizer= 'adam', 
    loss = 'categorical_crossentropy',
    metrics = ['acc',
               metrics.Precision(),
               metrics.Recall(),
               metrics.AUC()]
)

In [291]:
es = EarlyStopping(patience = 10,
                   monitor = 'val_loss',
                   mode = 'min')

lr = ReduceLROnPlateau(monitor = 'val_loss',
                       factor = 0.4,
                       patience = 2,
                       min_delta = 0.0001,
                       verbose = 1)

ck = ModelCheckpoint('../../Models/effnet_2.h5',
                    save_best_only = True,
                    monitor = 'val_acc',
                    mode = 'max')

In [ ]:
effnet_2 = model7.fit(train, 
                validation_data = val,
                epochs = 50,
                callbacks = [es, lr, ck],
                verbose = 1)

Epoch 1/50
1263/1263 [==============================] - 427s 301ms/step - loss: 1.7730 - acc: 0.3316 - precision_49: 0.5948 - recall_49: 0.1310 - auc_49: 0.7080 - val_loss: 1.4604 - val_acc: 0.4361 - val_precision_49: 0.7914 - val_recall_49: 0.1905 - val_auc_49: 0.8069 - lr: 0.0010
Epoch 2/50
1263/1263 [==============================] - 216s 171ms/step - loss: 1.4613 - acc: 0.4372 - precision_49: 0.7161 - recall_49: 0.2185 - auc_49: 0.8085 - val_loss: 1.3749 - val_acc: 0.4698 - val_precision_49: 0.7630 - val_recall_49: 0.2261 - val_auc_49: 0.8335 - lr: 0.0010
Epoch 3/50
1263/1263 [==============================] - 209s 165ms/step - loss: 1.3659 - acc: 0.4761 - precision_49: 0.7233 - recall_49: 0.2656 - auc_49: 0.8355 - val_loss: 1.3814 - val_acc: 0.4750 - val_precision_49: 0.7702 - val_recall_49: 0.2515 - val_auc_49: 0.8306 - lr: 0.0010
Epoch 4/50
1263/1263 [==============================] - 215s 170ms/step - loss: 1.3216 - acc: 0.4939 - precision_49: 0.7239 - recall_49: 0.2878 - auc_4

The second efficientnet model was stopped during it's training as the val_accuracy was not improving and doing worse than the custom sequential model performance (highest val_acc = 64.63%)

Moving forward, further analysis will be done on the custom CNN models.